<a href="https://colab.research.google.com/github/Shashank-Shukla/PyTorch-practice/blob/master/Pytorch_fully_Connected_network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
from torch import nn, optim
# All neural network modules, nn.Linear, nn.Conv2d, BatchNorm, Loss functions
# For all Optimization algorithms, SGD, Adam, etc.
from torch.nn import functional as f
# All functions that don't have any parameters ^
from torch.utils.data import DataLoader
# Gives easier dataset managment and creates mini batches
from torchvision import datasets, transforms
# Datasets has standard datasets we can import in a nice way
# Transformations we can perform on our dataset

import pandas as pd


In [2]:
# Creating a fully connected network
class NN(nn.Module):
  def __init__(self, input_size, no_classes):
    super(NN, self).__init__()
    self.fc1 = nn.Linear(input_size, 50) # 50 nodes
    self.fc2 = nn.Linear(50, no_classes)

  def forward(self, x):
    x = f.relu(self.fc1(x))
    x = self.fc2(x)
    return x
  

In [3]:
# set device as cuda if GPU is available else CPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Setting hyperparameters
input_size = 784
no_classes = 10
lr = 0.001
batch_size = 64
num_epochs = 20
train_dataset = datasets.MNIST('dataset/', train=True, transform=transforms.ToTensor(), download=True)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataset = datasets.MNIST('dataset/', train=True, transform=transforms.ToTensor(), download=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

# initialize network
model = NN(input_size, no_classes).to(device)

# Calc loss and optimizer
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(),lr=lr)


Extracting dataset/MNIST/raw/train-images-idx3-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/train-labels-idx1-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to dataset/MNIST/raw


Extracting dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to dataset/MNIST/raw
Processing...
Done!





/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:469: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
# Train the network
for epochs in range(num_epochs):
  for batch_index, (data, targets) in enumerate(train_loader):
    # data = images, targets = label of those images
    # Get data to cuda if possible (GPU present)
    data = data.to(device=device)
    targets = targets.to(device=device)
    # data.shape = torch.Size([64, 1, 28, 28])
    
    # Get data to correct shape
    data = data.reshape(data.shape[0],-1)
    # We'll keep the 1st dim as it is and flatten the rest. hence -1

    # Forward modelling the data
    scores = model(data)
    loss = loss_function(scores, targets)

    # Backpropagation
    optimizer.zero_grad()
    # So that it doesn't store the BackProp from previous iteration
    loss.backward()

    # Performing gradient descent or Adam step
    optimizer.step()


In [5]:
# Checking accuracy on training and test
def check_accuracy(loader, model):
  if loader.dataset.train:
    print("Checking accuracy on Training set")
  else:
    print("Checking accuracy on Testing set")
  num_correct = 0
  num_samples = 0
  model.eval()
  # We wanna set the model to evaluate mode

  with torch.no_grad():
    # Let torch know that we don't wanna calc gradient again
    for x, y in loader:
      x = x.to(device=device)
      y = y.to(device=device)
      x = x.reshape(x.shape[0], -1)

      scores = model(x)
      _, pred = scores.max(1) # returns value and index
      # We need index of max value for 2nd dim.
      num_correct += (pred == y).sum()
      num_samples += pred.size(0)

    accuracy = 100*float(num_correct)/num_samples
    print(f"Got {num_correct}/{num_samples} with accuracy = {accuracy:.2f}")
  model.train()
  return accuracy


In [6]:
check_accuracy(train_loader, model)
check_accuracy(test_loader, model)

Checking accuracy on Training set
Got 59756/60000 with accuracy = 99.59
Checking accuracy on Training set
Got 59756/60000 with accuracy = 99.59


99.59333333333333

In [7]:
torch.cuda.is_available()

True